In [1]:
# train_atari.py

import torch
import torch.backends.cudnn as cudnn  # Added import
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

from dqn.agent import DQNAgent
from dqn.replay_buffer import ReplayBuffer
from dqn.wrappers import make_atari, wrap_deepmind

# Enable cuDNN benchmark mode for optimized performance
cudnn.benchmark = True

# Set device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

env_name = 'PongNoFrameskip-v4'

# Create the Atari environment with the appropriate wrappers
env = make_atari(env_name)
env = wrap_deepmind(env, frame_stack=True, scale=False)  # Set scale=False since we're normalizing in the agent

# Verify the observation space
print("Observation Space:", env.observation_space)
print("Type:", type(env.observation_space))

# Initialize the replay buffer
replay_buffer = ReplayBuffer(size=100000)

# Increased batch size to better utilize GPU
batch_size = 128

# Initialize the DQN agent
agent = DQNAgent(
    observation_space=env.observation_space,
    action_space=env.action_space,
    replay_buffer=replay_buffer,
    use_double_dqn=True,
    lr=1e-4,
    batch_size=batch_size,
    gamma=0.99
)

# Training parameters
num_episodes = 1000
target_update_interval = 10  # Update target network every 10 episodes
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.1
rewards = []
losses = []

for episode in range(1, num_episodes + 1):
    state, _ = env.reset()
    total_reward = 0
    t = 0

    while True:
        t += 1

        # Epsilon-greedy action selection
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            action = agent.act(state)

        # Take a step in the environment
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Add experience to replay buffer
        agent.replay_buffer.add(state, action, reward, next_state, float(done))

        state = next_state
        total_reward += reward

        # Optimize the model
        loss = agent.optimise_td_loss()
        if loss is not None:
            losses.append(loss)

        if done:
            break

    # Decay epsilon after each episode
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    rewards.append(total_reward)

    # Update the target network at specified intervals
    if episode % target_update_interval == 0:
        agent.update_target_network()

    print(f"Episode {episode} - Total Reward: {total_reward} - Epsilon: {epsilon:.3f}")

# Plotting the reward curve
plt.figure(figsize=(10, 5))
plt.plot(rewards)
plt.title("Training Reward Curve")
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.show()

# Plotting the loss curve
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title("Training Loss Curve")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.show()

# Save the trained model
torch.save(agent.policy_network.state_dict(), "dqn_pong_model.pth")


Using device: cuda
Observation Space: Box(0, 255, (4, 84, 84), uint8)
Type: <class 'gymnasium.spaces.box.Box'>
Episode 1 - Total Reward: -21.0 - Epsilon: 0.995
Episode 2 - Total Reward: -21.0 - Epsilon: 0.990
Episode 3 - Total Reward: -21.0 - Epsilon: 0.985
Episode 4 - Total Reward: -21.0 - Epsilon: 0.980
Episode 5 - Total Reward: -21.0 - Epsilon: 0.975
Episode 6 - Total Reward: -21.0 - Epsilon: 0.970
Episode 7 - Total Reward: -19.0 - Epsilon: 0.966
Episode 8 - Total Reward: -20.0 - Epsilon: 0.961
Episode 9 - Total Reward: -21.0 - Epsilon: 0.956
Episode 10 - Total Reward: -20.0 - Epsilon: 0.951
Episode 11 - Total Reward: -19.0 - Epsilon: 0.946
Episode 12 - Total Reward: -20.0 - Epsilon: 0.942
Episode 13 - Total Reward: -21.0 - Epsilon: 0.937
Episode 14 - Total Reward: -21.0 - Epsilon: 0.932
Episode 15 - Total Reward: -18.0 - Epsilon: 0.928
Episode 16 - Total Reward: -21.0 - Epsilon: 0.923
Episode 17 - Total Reward: -21.0 - Epsilon: 0.918
Episode 18 - Total Reward: -20.0 - Epsilon: 0.91

KeyboardInterrupt: 